# 1. 환경 설정 및 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("=" * 60)
print("1️⃣ 환경 설정 및 라이브러리 설치")
print("=" * 60)

!pip install ultralytics pycocotools tqdm -q

import os
import json
import glob
import shutil
import random
import zipfile
from pathlib import Path
from tqdm import tqdm
import torch
from ultralytics import YOLO

print("✅ 라이브러리 설치 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

# 2. 데이터셋 설정


In [ ]:
from pathlib import Path

# ===============================
# 📂 사용자 지정 경로 설정 (수정 필수)
# ===============================
raw_base = Path(input("📁 이미지 폴더 경로를 입력하세요: ").strip())
label_base = Path(input("🏷️ 라벨 폴더 경로를 입력하세요: ").strip())

# 지원하는 이미지 확장자
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# ===============================
# 1️⃣ 재귀적으로 이미지/라벨 수집
# ===============================
images = [p for p in raw_base.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXTS]
labels = [p for p in label_base.rglob("*.json")] if label_base.exists() else []

print("\n" + "="*60)
print("📊 전체 개수 (재귀 탐색 결과)")
print("="*60)
print(f"🖼️ 이미지 파일 수 : {len(images)}개")
print(f"🏷️ 라벨(JSON) 파일 수 : {len(labels)}개")

# ===============================
# 2️⃣ 파일명(확장자 제외) 매칭 검사
# ===============================
img_stems = {p.stem for p in images}
lbl_stems = {p.stem for p in labels}

only_img = sorted(list(img_stems - lbl_stems))[:10]
only_lbl = sorted(list(lbl_stems - img_stems))[:10]

print("\n🔍 매칭 상태")
print(f" - 라벨이 없는 이미지: {len(img_stems - lbl_stems)}개 (예시: {only_img})")
print(f" - 이미지가 없는 라벨: {len(lbl_stems - img_stems)}개 (예시: {only_lbl})")


# 3. 학습 설정

In [ ]:
print("\n" + "=" * 60)
print("3️⃣ 학습 설정")
print("=" * 60)

from argparse import Namespace
import os, torch

# ===============================
# ⚙️ 사용자 입력 기반 설정
# ===============================
config_path = input("📄 dataset.yaml 파일 경로를 입력하세요: ").strip()

args = Namespace(
    epoch=100,
    project="yolo11-downsampling",
    name="experiment",
    config=config_path,
    cuda="0",
    resume=False,
    resume_path=""
)

# ===============================
# 🚀 디바이스 설정
# ===============================
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda
os.environ["WANDB_MODE"] = "disabled"

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"✅ 디바이스: {device}")
print(f"✅ Epochs: {args.epoch}")
print(f"✅ 프로젝트: {args.project}/{args.name}")
print(f"✅ 데이터 설정 파일: {args.config}")


# 4. YOLO11 표준 디렉토리 구조 생성


In [ ]:
print("\n" + "=" * 60)
print("4️⃣ YOLO11 표준 디렉토리 구조 생성 (재귀탐색/확장자 다중/안전 Stratify)")
print("=" * 60)

import os, json, shutil, pickle
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 경로 설정
dataset_root = Path("") # Dataset 경로를 기입해주세요.
raw_base = Path("") # images 경로를 기입해주세요. 
label_base = Path("") #labels 경로를 기입해주세요. 

# 결과 디렉토리 생성
for split in ["train", "val", "test"]:
    (dataset_root / "images" / split).mkdir(parents=True, exist_ok=True)
    (dataset_root / "labels" / split).mkdir(parents=True, exist_ok=True)

# 이미지 확장자
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# 1️⃣ 재귀로 모든 이미지 수집
print("📂 재귀로 이미지 수집 중...")
images = [p for p in raw_base.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXTS]
print(f"✅ 총 {len(images)}개 이미지 수집 완료")

if len(images) == 0:
    print("❌ 이미지가 0개입니다. 아래를 확인하세요.")
    print("  - raw_base 경로가 맞나요? =>", raw_base)
    print("  - 실제 확장자가 .jpg 외에 .JPG/.png/.jpeg 인가요?")
    print("  - 이미지가 더 깊은 하위폴더에 있나요? (이 코드는 재귀라면 OK)")
    raise SystemExit

# (img_path, img_file, subdir) 형태로 정리
all_image_info = [(str(p), p.name, str(p.parent.relative_to(raw_base))) for p in images]

# 2️⃣ 라벨 추출 (shapes 비면 Normal로 인식)
print("\n🔍 라벨 추출 중...")
all_image_paths, all_labels, all_label_paths = [], [], []
missing_label = 0

for img_path, img_file, subdir in tqdm(all_image_info, desc="라벨 로딩"):
    # labels/<subdir>/<same_name>.json 을 기대
    if subdir == ".":
        json_path = label_base / (Path(img_file).with_suffix(".json").name)
    else:
        json_path = label_base / subdir / (Path(img_file).with_suffix(".json").name)

    if json_path.exists():
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # ✅ 핵심 수정: shapes가 비면 Normal로 간주
            if isinstance(data, dict):
                shapes = data.get("shapes", [])
                if len(shapes) == 0:
                    label = "Normal"  # 정상(객체 없음)
                else:
                    label = shapes[0].get("label", "정보부족")
            else:
                label = "정보부족"

        except Exception:
            label = "정보부족"
    else:
        label = "정보부족"
        missing_label += 1
        json_path = None  # 복사 시 스킵

    all_image_paths.append(img_path)
    all_labels.append(label)
    all_label_paths.append(str(json_path) if json_path else None)

print(f"✅ 라벨 로드 완료 (총 {len(all_labels)}개) / 라벨 누락: {missing_label}개")

# 3️⃣ Stratify 여부 판단
cls_counts = Counter(all_labels)
can_stratify = len(cls_counts) >= 2 and min(cls_counts.values()) >= 2

train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1
test_size = 1 - train_ratio
val_size = val_ratio / (val_ratio + test_ratio)

print("\n📊 데이터 분할 중...")
if can_stratify:
    strat = all_labels
    print(f"  - Stratify 적용 (클래스 분포: {dict(cls_counts)})")
else:
    strat = None
    print(f"  - Stratify 미적용 (클래스 분포: {dict(cls_counts)})")

train_paths, temp_paths, train_labels, temp_labels, train_lblpaths, temp_lblpaths = train_test_split(
    all_image_paths, all_labels, all_label_paths,
    test_size=test_size, stratify=strat, random_state=42
)

if can_stratify:
    strat2 = temp_labels
else:
    strat2 = None

val_paths, test_paths, val_labels, test_labels, val_lblpaths, test_lblpaths = train_test_split(
    temp_paths, temp_labels, temp_lblpaths,
    test_size=(1 - val_size), stratify=strat2, random_state=42
)

print(f"  - Train: {len(train_paths)}개")
print(f"  - Val  : {len(val_paths)}개")
print(f"  - Test : {len(test_paths)}개")

# 4️⃣ 복사
print("\n🔄 이미지 및 라벨 복사 중...")

def copy_split(paths, lblpaths, split):
    for p, lp in tqdm(list(zip(paths, lblpaths)), desc=f"{split} 세트 복사", total=len(paths)):
        filename = Path(p).name
        dst_img = dataset_root / "images" / split / filename
        shutil.copy2(p, dst_img)
        if lp and Path(lp).exists():
            dst_json = dataset_root / "labels" / split / (Path(filename).with_suffix(".json").name)
            shutil.copy2(lp, dst_json)

copy_split(train_paths, train_lblpaths, "train")
copy_split(val_paths,   val_lblpaths,   "val")
copy_split(test_paths,  test_lblpaths,  "test")

print("\n✅ Stratify 기반(가능 시) 분할 + 복사 완료")

# 5️⃣ 매핑 정보 저장
image_to_subdir = {}
for paths, split in [(train_paths, "train"), (val_paths, "val"), (test_paths, "test")]:
    for p in paths:
        p = Path(p)
        subdir = str(p.parent.relative_to(raw_base))
        image_to_subdir[f"{split}/{p.name}"] = subdir

with open(dataset_root / "image_to_subdir.pkl", "wb") as f:
    pickle.dump(image_to_subdir, f)

print("✅ 매핑 정보 저장 완료 (image_to_subdir.pkl)")


# 5. JSON → YOLO11 라벨 변환

In [ ]:
print("\n" + "=" * 60)
print("5️⃣ JSON → YOLO11 라벨 변환")
print("=" * 60)

import os, json, pickle
from tqdm import tqdm

# JSON이 아예 없을 때도 Normal(빈 .txt)로 만들지 여부
TREAT_JSON_MISSING_AS_NORMAL = True

# 클래스 매핑 (이미 위 셀에서 정의한 걸 그대로 사용해도 됨)
class_name_to_number = {
    'ConcreteCrack': '0',
    'Spalling': '1',
    'Efflorescene': '2',
    'Leak': '3',
    'Exposure': '4',
    'AspaltCrack': '5',
    'Pothole': '6',
    'SteelDamage': '7',
}

def process_coordinates(points, img_width=512, img_height=512):
    processed_coords = []
    for point in points:
        if isinstance(point, list) and len(point) >= 2:
            x = float(point[0]) / img_width
            y = float(point[1]) / img_height
            processed_coords.append(f"{x:.6f} {y:.6f}")
    return ' '.join(processed_coords)

def convert_json_to_yolo(json_path, output_path):
    """
    - shapes가 비면 Normal(객체 0개)로 간주 → 빈 .txt 생성
    - shapes가 있으면 각 polygon을 YOLO 세그 형식으로 기록
    - 성공 시 True, 변환할 객체 없으면 False 반환
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_output = []
        img_width  = data.get('imageWidth',  512)
        img_height = data.get('imageHeight', 512)
        shapes     = data.get('shapes', [])

        # ★ Normal(객체 없음): shapes가 비면 빈 .txt 생성 후 True 반환
        if not shapes:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write("")   # 내용 없는 .txt
            return True

        for shape in shapes:
            label = shape.get('label', 'Unknown')
            class_number = class_name_to_number.get(label, None)
            if class_number is None:
                continue

            points = shape.get('points', [])
            if not points or len(points) < 3:
                continue

            coordinates = process_coordinates(points, img_width, img_height)
            if coordinates:
                txt_output.append(f"{class_number} {coordinates}")

        if txt_output:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(txt_output))
            return True

        # 객체가 제대로 기록되지 않은 경우
        return False

    except Exception:
        # JSON 파싱 실패 등
        return False

# ===== 매핑 정보 로드 =====
with open(f"{dataset_root}/image_to_subdir.pkl", 'rb') as f:
    image_to_subdir = pickle.load(f)

label_base = "" # labels 경로 기입해주세요. 

print("🔄 라벨 변환 중...")
conversion_stats = {'train': 0, 'val': 0, 'test': 0}

for split in ['train', 'val', 'test']:
    img_folder   = f"{dataset_root}/images/{split}"
    label_folder = f"{dataset_root}/labels/{split}"

    # 이미지 목록(.jpg만) → 필요하면 확장자 추가 가능
    image_files_in_split = [f for f in os.listdir(img_folder) if f.lower().endswith('.jpg')]

    for img_file in tqdm(image_files_in_split, desc=f"{split} 라벨 변환"):
        # 원래 하위 폴더(레이블 위치) 추적
        subdir = image_to_subdir.get(f"{split}/{img_file}", None)
        if subdir is None:
            continue

        img_name = os.path.splitext(img_file)[0]
        json_path = os.path.join(label_base, subdir, f"{img_name}.json")
        txt_path  = os.path.join(label_folder, f"{img_name}.txt")

        if os.path.exists(json_path):
            if convert_json_to_yolo(json_path, txt_path):
                conversion_stats[split] += 1
        else:
            # ★ JSON 자체가 없을 때도 Normal(빈 .txt)로 취급하고 싶다면:
            if TREAT_JSON_MISSING_AS_NORMAL:
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write("")  # 빈 .txt 생성
                conversion_stats[split] += 1
            # 아니면 스킵

print("\n✅ 라벨 변환 완료")
for split, count in conversion_stats.items():
    total = len([f for f in os.listdir(f"{dataset_root}/images/{split}") if f.lower().endswith('.jpg')])
    rate = (count / total * 100) if total > 0 else 0
    print(f"  - {split}: {count}/{total}개 ({rate:.1f}%)")


# 6. YAML 설정 파일 생성

In [ ]:
print("\n" + "=" * 60)
print("6️⃣ YAML 설정 파일 생성")
print("=" * 60)

dataset_root = "" # 데이터셋 경로 기입해주세요. 

yaml_content = f"""# YOLO11 Segmentation Dataset
path: {dataset_root}
train: images/train
val: images/val
test: images/test

names:
  0: ConcreteCrack
  1: Spalling
  2: Efflorescene
  3: Leak
  4: Exposure
  5: AspaltCrack
  6: Pothole
  7: SteelDamage

nc: 8
"""

with open(args.config, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"✅ YAML 파일 생성: {args.config}")

# 7. 데이터셋 검증

In [ ]:
print("\n" + "=" * 60)
print("7️⃣ 데이터셋 검증")
print("=" * 60)


# 경로 설정
dataset_root = Path("") # dataset 경로를 입력해주세요. 
raw_base = Path("") # images 경로를 입력해주세요. 
label_base = Path("") # labels 경로를 입력해주세요. 

# 클래스 매핑 (이미 위 셀에서 정의한 걸 그대로 사용해도 됨)
class_name_to_number = {
    'ConcreteCrack': '0',
    'Spalling': '1',
    'Efflorescene': '2',
    'Leak': '3',
    'Exposure': '4',
    'AspaltCrack': '5',
    'Pothole': '6',
    'SteelDamage': '7',
}

for split in ['train', 'val', 'test']:
    img_folder = f"{dataset_root}/images/{split}"
    label_folder = f"{dataset_root}/labels/{split}"

    img_count = len([f for f in os.listdir(img_folder) if f.endswith('.jpg')])
    label_count = len([f for f in os.listdir(label_folder) if f.endswith('.txt')])

    match_rate = (label_count / img_count * 100) if img_count > 0 else 0
    print(f"{split:6s}: 이미지 {img_count:4d}개, 라벨 {label_count:4d}개 (매칭률: {match_rate:.1f}%)")

sample_labels = glob.glob(f'{dataset_root}/labels/train/*.txt')
if sample_labels:
    with open(sample_labels[0], 'r') as f:
        lines = f.readlines()
        print(f"\n📝 샘플 라벨: {os.path.basename(sample_labels[0])}")
        print(f"   - 객체 수: {len(lines)}")
        if lines:
            parts = lines[0].strip().split()
            print(f"   - 클래스: {parts[0]}, 좌표점: {(len(parts)-1)//2}개")

print(f"\n📊 클래스별 통계:")
class_counts = {name: 0 for name in class_name_to_number.keys()}

for split in ['train', 'val', 'test']:
    for txt_file in os.listdir(f"{dataset_root}/labels/{split}"):
        if txt_file.endswith('.txt'):
            with open(f"{dataset_root}/labels/{split}/{txt_file}", 'r') as f:
                for line in f:
                    class_id = line.strip().split()[0]
                    for name, cid in class_name_to_number.items():
                        if cid == class_id:
                            class_counts[name] += 1
                            break

total_objects = sum(class_counts.values())
for class_name, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    if count > 0:
        percentage = (count / total_objects * 100)
        print(f"  {class_name:15s}: {count:4d}개 ({percentage:5.1f}%)")

# 8. YOLO11 모델 학습

In [ ]:
import glob, os

dataset_root = Path("") # dataset 경로를 입력해주세요. 

# dataset_root는 코드에서 이미 정의되어 있음
for cache_file in glob.glob(f"{dataset_root}/**/*.cache", recursive=True):
    print(f"🗑 삭제: {cache_file}")
    os.remove(cache_file)

# T4 기준, 15분 내외로 학습 소요
print("\n" + "=" * 60)
print("8️⃣ YOLO11 모델 학습")
print("=" * 60)

MODEL = "yolo11m-seg.pt" # 원하는 모델로 변경하여 기입

if args.resume and os.path.exists(args.resume_path):
    print(f"🔄 체크포인트 재개: {args.resume_path}")
    model = YOLO(args.resume_path)
else:
    print(f"🆕 새 모델 로드: {MODEL}")
    model = YOLO(MODEL)

model.to(device)

print(f"\n🚀 학습 시작...")
results = model.train(
    project=args.project,
    name=args.name,
    data=args.config,
    epochs=args.epoch,
    imgsz=512,
    batch=32, # 기존의 코드는 16으로 했는데, VRAM이 남아서 32로 진행해도 괜찮아 보입니다.
    device=device,
    optimizer="auto",
    val=True,
    resume=args.resume,
    verbose=True,
    patience=100,
    save=True,
    plots=True,
)

print(f"\n✅ 학습 완료!")
print(f"📁 결과: {args.project}/{args.name}/")

# 9. 최적 모델 검증

In [ ]:
print("\n" + "=" * 60)
print("9️⃣ 최적 모델 검증")
print("=" * 60)

best_model_path = f"{args.project}/{args.name}/weights/best.pt"

if os.path.exists(best_model_path):
    print(f"📦 최적 모델 로드: {best_model_path}")
    best_model = YOLO(best_model_path)

    print("\n📊 검증 세트 평가...")
    val_results = best_model.val(
        data=args.config,
        split='val',
        imgsz=512,
        batch=16,
        device=device,
    )

    print("\n✅ 검증 완료!")
    print(f"   - Box mAP50:     {val_results.box.map50:.4f}")
    print(f"   - Box mAP50-95:  {val_results.box.map:.4f}")
    print(f"   - Mask mAP50:    {val_results.seg.map50:.4f}")
    print(f"   - Mask mAP50-95: {val_results.seg.map:.4f}")
else:
    print(f"⚠️  최적 모델을 찾을 수 없음: {best_model_path}")

# 10. 테스트 세트 추론

In [ ]:
import numpy as np
import cv2
import os, glob
from tqdm import tqdm
from sklearn.metrics import jaccard_score

test_images_folder = f"{dataset_root}/images/test"
test_labels_folder = f"{dataset_root}/labels/test"   # YOLO txt GT 폴더
test_images = glob.glob(os.path.join(test_images_folder, "*.jpg"))

CLASS_NAMES = [
    "ConcreteCrack",
    "Spalling",
    "Efflorescene",
    "Leak",
    "Exposure",
    "AspaltCrack",
    "Pothole",
    "SteelDamage"
]
NUM_CLASSES = len(CLASS_NAMES)


# -----------------------------
# 보조 함수: YOLO polygon → mask 변환
# -----------------------------
def yolo_txt_to_mask(txt_path, img_shape, num_classes):
    """YOLO polygon 형식 txt 파일을 픽셀 마스크로 변환"""
    h, w = img_shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    if not os.path.exists(txt_path):
        return mask

    with open(txt_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:
            continue
        cls_id = int(parts[0])
        coords = np.array(list(map(float, parts[1:])))
        xs = (coords[0::2] * w).astype(np.int32)
        ys = (coords[1::2] * h).astype(np.int32)
        poly = np.stack([xs, ys], axis=1)
        cv2.fillPoly(mask, [poly], color=cls_id + 1)
    return mask

In [ ]:
# -----------------------------
# IoU 계산
# -----------------------------
ious_all = []
ious_per_class = {cls: [] for cls in CLASS_NAMES}

print(f"🔍 총 {len(test_images)}개 이미지 평가 중...")

for img_path in tqdm(test_images, desc="IoU 계산"):
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    gt_label_path = os.path.join(test_labels_folder, f"{img_name}.txt")

    # 이미지 크기
    img = cv2.imread(img_path)
    if img is None:
        continue
    h, w, _ = img.shape

    # 1️⃣ GT mask 생성
    gt_mask = yolo_txt_to_mask(gt_label_path, (h, w, 3), NUM_CLASSES)

    # 2️⃣ YOLO 예측 수행
    results = best_model.predict(
        source=img_path,
        imgsz=512,
        conf=0.25,
        iou=0.5,
        device=device,
        save=False,
        verbose=False
    )

    # 3️⃣ 예측 마스크 생성
    pred_mask = np.zeros((h, w), dtype=np.uint8)
    if results[0].masks is not None:
        for m, c in zip(results[0].masks.data, results[0].boxes.cls):
            cls_id = int(c.item()) + 1
            mask = (m.cpu().numpy() * 255).astype(np.uint8)
            mask_resized = cv2.resize(mask, (w, h))
            pred_mask[mask_resized > 127] = cls_id

    # 4️⃣ IoU 계산
    gt_flat = gt_mask.flatten()
    pred_flat = pred_mask.flatten()
    iou_mean = jaccard_score(gt_flat, pred_flat, average="macro", zero_division=0)
    ious_all.append(iou_mean)

    for idx, cls_name in enumerate(CLASS_NAMES, start=1):
        gt_bin = (gt_mask == idx).astype(np.uint8)
        pred_bin = (pred_mask == idx).astype(np.uint8)
        if np.sum(gt_bin) == 0 and np.sum(pred_bin) == 0:
            continue
        iou_cls = jaccard_score(gt_bin.flatten(), pred_bin.flatten(), average="binary", zero_division=0)
        ious_per_class[cls_name].append(iou_cls)



In [ ]:
# -----------------------------
# 결과 요약 출력
# -----------------------------
mean_iou = np.mean(ious_all)
median_iou = np.median(ious_all)
std_iou = np.std(ious_all)

print("\n" + "=" * 60)
print("📊 세그멘테이션 IoU 결과 요약")
print("=" * 60)
print(f"📌 Mean IoU:   {mean_iou:.4f}")
print(f"📌 Median IoU: {median_iou:.4f}")
print(f"📌 Std Dev:    {std_iou:.4f}")

print("\n📈 클래스별 평균 IoU")
for cls, vals in ious_per_class.items():
    if len(vals) > 0:
        print(f" - {cls:15s}: {np.mean(vals):.4f}")
    else:
        print(f" - {cls:15s}: 데이터 없음")

print("\n✅ 세그멘테이션 IoU 계산 완료!")

# 11. 추론 결과 시각화

In [ ]:
print("\n" + "=" * 60)
print("1️⃣1️⃣ 추론 결과 시각화")
print("=" * 60)

import matplotlib.pyplot as plt
from PIL import Image

result_images = glob.glob(f"{output_folder}/test_results/*.jpg")[:5]

if result_images:
    fig, axes = plt.subplots(1, len(result_images), figsize=(20, 4))
    if len(result_images) == 1:
        axes = [axes]

    for idx, img_path in enumerate(result_images):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(os.path.basename(img_path), fontsize=8)

    plt.tight_layout()
    plt.savefig(f"{output_folder}/visualization.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✅ 시각화 저장: {output_folder}/visualization.png")
else:
    print("⚠️  추론 결과를 찾을 수 없음")

# 12. 학습 결과 요약

In [ ]:
print("\n" + "=" * 60)
print("1️⃣2️⃣ 학습 결과 요약")
print("=" * 60)

results_png = f"{args.project}/{args.name}/results.png"
confusion_matrix_png = f"{args.project}/{args.name}/confusion_matrix.png"

if os.path.exists(results_png) or os.path.exists(confusion_matrix_png):
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    if os.path.exists(results_png):
        img1 = Image.open(results_png)
        axes[0].imshow(img1)
        axes[0].set_title('Training Results', fontweight='bold')
        axes[0].axis('off')

    if os.path.exists(confusion_matrix_png):
        img2 = Image.open(confusion_matrix_png)
        axes[1].imshow(img2)
        axes[1].set_title('Confusion Matrix', fontweight='bold')
        axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"{args.project}/{args.name}/training_summary.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✅ 학습 요약: {args.project}/{args.name}/training_summary.png")

print("\n" + "=" * 60)
print("🎉 모든 작업 완료!")
print("=" * 60)
print(f"\n📂 주요 결과물:")
print(f"   1. 데이터셋: {dataset_root}/")
print(f"   2. 최적 모델: {best_model_path}")
print(f"   3. 학습 결과: {args.project}/{args.name}/")
print(f"   4. 추론 결과: {output_folder}/test_results/")

# 13. Google Drive 마운트 및 백업

In [ ]:
print("\n" + "=" * 60)
print("1️⃣3️⃣ Google Drive 백업")
print("=" * 60)

from google.colab import drive
import shutil
from datetime import datetime

# Google Drive 마운트
print("🔗 Google Drive 마운트 중...")
drive.mount('/content/drive')
print("✅ Google Drive 마운트 완료")

# 백업 디렉토리 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_root = f"/YOLO11_Backup_{timestamp}" # 저장할 경로를 기입해주세요. 
os.makedirs(backup_root, exist_ok=True)

print(f"\n📁 백업 디렉토리: {backup_root}")

In [ ]:
# ========================================
# 1. 학습 결과 백업
# ========================================
print("\n" + "=" * 60)
print("1️⃣ 학습 결과 백업")
print("=" * 60)

class_name_to_number = {
    'ConcreteCrack': '0',
    'Spalling': '1',
    'Efflorescene': '2',
    'Leak': '3',
    'Exposure': '4',
    'AspaltCrack': '5',
    'Pothole': '6',
    'SteelDamage': '7',
}


source_project = f"{args.project}/{args.name}"
dest_project = f"{backup_root}/training_results"

if os.path.exists(source_project):
    print(f"📦 복사 중: {source_project}")
    shutil.copytree(source_project, dest_project, dirs_exist_ok=True)
    print(f"✅ 저장 완료: {dest_project}")

    # 주요 파일 확인
    important_files = [
        'weights/best.pt',
        'weights/last.pt',
        'results.png',
        'confusion_matrix.png',
        'results.csv'
    ]

    print("\n주요 파일 확인:")
    for file in important_files:
        file_path = os.path.join(dest_project, file)
        if os.path.exists(file_path):
            size = os.path.getsize(file_path) / (1024**2)  # MB
            print(f"  ✓ {file}: {size:.2f} MB")
        else:
            print(f"  ✗ {file}: 없음")
else:
    print(f"⚠️  학습 결과를 찾을 수 없습니다: {source_project}")

# ========================================
# 2. 데이터셋 정보 백업 (구조만)
# ========================================
print("\n" + "=" * 60)
print("2️⃣ 데이터셋 정보 백업")
print("=" * 60)

dataset_info_dir = f"{backup_root}/dataset_info"
os.makedirs(dataset_info_dir, exist_ok=True)

# YAML 파일 복사
if os.path.exists(args.config):
    shutil.copy2(args.config, f"{dataset_info_dir}/dataset.yaml")
    print(f"✅ YAML 파일 복사 완료")

# 매핑 정보 복사
mapping_file = f"{dataset_root}/image_to_subdir.pkl"
if os.path.exists(mapping_file):
    shutil.copy2(mapping_file, f"{dataset_info_dir}/image_to_subdir.pkl")
    print(f"✅ 매핑 파일 복사 완료")

# 데이터셋 통계 저장
stats_file = f"{dataset_info_dir}/dataset_statistics.txt"
with open(stats_file, 'w', encoding='utf-8') as f:
    f.write("=" * 60 + "\n")
    f.write("YOLO11 데이터셋 통계\n")
    f.write("=" * 60 + "\n\n")

    f.write(f"생성 시간: {timestamp}\n\n")

    f.write("데이터 분할:\n")
    for split in ['train', 'val', 'test']:
        img_count = len([f for f in os.listdir(f"{dataset_root}/images/{split}") if f.endswith('.jpg')])
        label_count = len([f for f in os.listdir(f"{dataset_root}/labels/{split}") if f.endswith('.txt')])
        f.write(f"  - {split:6s}: 이미지 {img_count:4d}개, 라벨 {label_count:4d}개\n")

    f.write("\n클래스별 통계:\n")
    class_counts = {name: 0 for name in class_name_to_number.keys()}

    for split in ['train', 'val', 'test']:
        for txt_file in os.listdir(f"{dataset_root}/labels/{split}"):
            if txt_file.endswith('.txt'):
                with open(f"{dataset_root}/labels/{split}/{txt_file}", 'r') as tf:
                    for line in tf:
                        parts = line.strip().split()
                        if parts:
                            class_id = parts[0]
                            for name, cid in class_name_to_number.items():
                                if cid == class_id:
                                    class_counts[name] += 1
                                    break

    total_objects = sum(class_counts.values())
    for class_name, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
        if count > 0:
            percentage = (count / total_objects * 100)
            f.write(f"  - {class_name:15s}: {count:4d}개 ({percentage:5.1f}%)\n")

    f.write(f"\n총 객체 수: {total_objects}개\n")

print(f"✅ 데이터셋 통계 저장 완료: dataset_statistics.txt")

# ========================================
# 3. 추론 결과 백업 (있는 경우)
# ========================================
print("\n" + "=" * 60)
print("3️⃣ 추론 결과 백업")
print("=" * 60)

prediction_source = f"{args.project}/{args.name}/predictions"
prediction_dest = f"{backup_root}/predictions"

if os.path.exists(prediction_source):
    print(f"📦 복사 중: {prediction_source}")
    shutil.copytree(prediction_source, prediction_dest, dirs_exist_ok=True)
    print(f"✅ 저장 완료: {prediction_dest}")
else:
    print(f"⚠️  추론 결과가 없습니다")

# ========================================
# 4. 샘플 이미지 및 라벨 백업
# ========================================
print("\n" + "=" * 60)
print("4️⃣ 샘플 데이터 백업")
print("=" * 60)

sample_dir = f"{backup_root}/samples"
os.makedirs(sample_dir, exist_ok=True)

# 각 split에서 5개씩 샘플 복사
for split in ['train', 'val', 'test']:
    split_sample_dir = f"{sample_dir}/{split}"
    os.makedirs(f"{split_sample_dir}/images", exist_ok=True)
    os.makedirs(f"{split_sample_dir}/labels", exist_ok=True)

    img_files = [f for f in os.listdir(f"{dataset_root}/images/{split}") if f.endswith('.jpg')][:5]

    for img_file in img_files:
        # 이미지 복사
        src_img = f"{dataset_root}/images/{split}/{img_file}"
        dst_img = f"{split_sample_dir}/images/{img_file}"
        shutil.copy2(src_img, dst_img)

        # 라벨 복사
        txt_file = img_file.replace('.jpg', '.txt')
        src_label = f"{dataset_root}/labels/{split}/{txt_file}"
        dst_label = f"{split_sample_dir}/labels/{txt_file}"

        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"✅ {split}: {len(img_files)}개 샘플 복사 완료")

# ========================================
# 5. 학습 설정 저장
# ========================================
print("\n" + "=" * 60)
print("5️⃣ 학습 설정 저장")
print("=" * 60)

config_file = f"{backup_root}/training_config.txt"
with open(config_file, 'w', encoding='utf-8') as f:
    f.write("=" * 60 + "\n")
    f.write("YOLOv11 학습 설정\n")
    f.write("=" * 60 + "\n\n")

    f.write(f"생성 시간: {timestamp}\n\n")

    f.write("학습 파라미터:\n")
    f.write(f"  - 모델: yolov11s-seg.pt\n")
    f.write(f"  - Epochs: {args.epoch}\n")
    f.write(f"  - Batch size: 16\n")
    f.write(f"  - Image size: 512\n")
    f.write(f"  - Device: {device}\n")
    f.write(f"  - Optimizer: auto (AdamW)\n")
    f.write(f"  - Patience: 100\n\n")

    f.write("프로젝트 정보:\n")
    f.write(f"  - Project: {args.project}\n")
    f.write(f"  - Name: {args.name}\n")
    f.write(f"  - Config: {args.config}\n\n")

    f.write("클래스 정의:\n")
    for name, cid in sorted(class_name_to_number.items(), key=lambda x: x[1]):
        f.write(f"  - {cid}: {name}\n")

print(f"✅ 학습 설정 저장 완료: training_config.txt")

# ========================================
# 6. README 생성
# ========================================
print("\n" + "=" * 60)
print("6️⃣ README 생성")
print("=" * 60)

readme_file = f"{backup_root}/README.md"
with open(readme_file, 'w', encoding='utf-8') as f:
    f.write("# YOLOv11 Segmentation 학습 결과\n\n")
    f.write(f"생성 일시: {timestamp}\n\n")

    f.write("## 📁 디렉토리 구조\n\n")
    f.write("```\n")
    f.write("YOLOv11_Backup_{timestamp}/\n")
    f.write("├── training_results/          # 학습 결과\n")
    f.write("│   ├── weights/\n")
    f.write("│   │   ├── best.pt           # 최적 모델\n")
    f.write("│   │   └── last.pt           # 마지막 모델\n")
    f.write("│   ├── results.png           # 학습 곡선\n")
    f.write("│   ├── confusion_matrix.png  # Confusion Matrix\n")
    f.write("│   └── results.csv           # 상세 결과\n")
    f.write("├── dataset_info/              # 데이터셋 정보\n")
    f.write("│   ├── dataset.yaml          # YAML 설정\n")
    f.write("│   ├── image_to_subdir.pkl   # 매핑 정보\n")
    f.write("│   └── dataset_statistics.txt # 통계\n")
    f.write("├── predictions/               # 추론 결과 (있는 경우)\n")
    f.write("├── samples/                   # 샘플 데이터\n")
    f.write("│   ├── train/\n")
    f.write("│   ├── val/\n")
    f.write("│   └── test/\n")
    f.write("├── training_config.txt        # 학습 설정\n")
    f.write("└── README.md                  # 이 파일\n")
    f.write("```\n\n")

    f.write("## 📊 데이터셋 정보\n\n")
    for split in ['train', 'val', 'test']:
        img_count = len([f for f in os.listdir(f"{dataset_root}/images/{split}") if f.endswith('.jpg')])
        label_count = len([f for f in os.listdir(f"{dataset_root}/labels/{split}") if f.endswith('.txt')])
        match_rate = (label_count / img_count * 100) if img_count > 0 else 0
        f.write(f"- **{split.capitalize()}**: {img_count}개 이미지, {label_count}개 라벨 ({match_rate:.1f}%)\n")

    f.write("\n## 🎯 클래스 정보\n\n")
    f.write("| ID | 클래스명 |\n")
    f.write("|:--:|:--------|\n")
    for name, cid in sorted(class_name_to_number.items(), key=lambda x: x[1]):
        f.write(f"| {cid} | {name} |\n")

    f.write("\n## 🚀 학습 설정\n\n")
    f.write(f"- 모델: YOLOv11s-seg\n")
    f.write(f"- Epochs: {args.epoch}\n")
    f.write(f"- Batch size: 16\n")
    f.write(f"- Image size: 512×512\n")
    f.write(f"- Device: {device}\n")

    f.write("\n## 📝 재현 방법\n\n")
    f.write("### 1. 모델 로드\n")
    f.write("```python\n")
    f.write("from ultralytics import YOLO\n\n")
    f.write("# 최적 모델 로드\n")
    f.write("model = YOLO('training_results/weights/best.pt')\n")
    f.write("```\n\n")

    f.write("### 2. 추론\n")
    f.write("```python\n")
    f.write("# 이미지 추론\n")
    f.write("results = model.predict(\n")
    f.write("    source='path/to/image.jpg',\n")
    f.write("    imgsz=512,\n")
    f.write("    conf=0.25,\n")
    f.write("    device='cuda'\n")
    f.write(")\n")
    f.write("```\n\n")

    f.write("### 3. 추가 학습 (Fine-tuning)\n")
    f.write("```python\n")
    f.write("# 학습 재개\n")
    f.write("model = YOLO('training_results/weights/best.pt')\n")
    f.write("model.train(\n")
    f.write("    data='dataset_info/dataset.yaml',\n")
    f.write("    epochs=50,\n")
    f.write("    resume=True\n")
    f.write(")\n")
    f.write("```\n\n")

    f.write("## 📧 문의\n\n")
    f.write("이 모델에 대한 문의사항은 프로젝트 담당자에게 연락하세요.\n")

print(f"✅ README 생성 완료: README.md")

# ========================================
# 7. 백업 요약
# ========================================
print("\n" + "=" * 60)
print("📊 백업 요약")
print("=" * 60)

def get_dir_size(path):
    """디렉토리 크기 계산 (MB)"""
    total = 0
    try:
        for dirpath, dirnames, filenames in os.walk(path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                if os.path.exists(fp):
                    total += os.path.getsize(fp)
    except:
        pass
    return total / (1024**2)

print(f"\n📁 백업 위치: {backup_root}\n")

backup_items = [
    ('training_results', '학습 결과'),
    ('dataset_info', '데이터셋 정보'),
    ('predictions', '추론 결과'),
    ('samples', '샘플 데이터')
]

total_size = 0
for item, desc in backup_items:
    item_path = f"{backup_root}/{item}"
    if os.path.exists(item_path):
        size = get_dir_size(item_path)
        total_size += size
        print(f"  ✓ {desc:15s}: {size:7.2f} MB")
    else:
        print(f"  - {desc:15s}: 없음")

# 설정 파일 크기
config_size = (os.path.getsize(config_file) + os.path.getsize(readme_file)) / (1024**2)
total_size += config_size

print(f"  ✓ {'설정 파일':15s}: {config_size:7.2f} MB")
print(f"  {'='*15}   {'='*7}")
print(f"  {'총 크기':15s}: {total_size:7.2f} MB")

print("\n" + "=" * 60)
print("✅ 모든 백업 완료!")
print("=" * 60)
print(f"\n📂 Google Drive에서 확인:")
print(f"   {backup_root.replace('/content/drive/RESULT/', '')}") # 경로 수정
print("\n💡 Tip: 백업된 폴더를 압축하면 더 쉽게 공유할 수 있습니다.")